In [2]:
%load_ext autoreload
%autoreload 2

In [142]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [6]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-02-13 04:00:00+00:00


In [144]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [135]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-02-12 23:23:46,340 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-12 23:23:46,352 INFO: Initializing external client
2025-02-12 23:23:46,352 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-12 23:23:47,450 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551


In [136]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.50s) 


In [137]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,22,0.0,2025-02-13 04:00:00+00:00
1,131,0.0,2025-02-13 04:00:00+00:00
2,36,0.0,2025-02-13 04:00:00+00:00
3,83,0.0,2025-02-13 04:00:00+00:00
4,152,1.0,2025-02-13 04:00:00+00:00
...,...,...,...
251,173,0.0,2025-02-13 04:00:00+00:00
252,229,61.0,2025-02-13 04:00:00+00:00
253,208,0.0,2025-02-13 04:00:00+00:00
254,45,4.0,2025-02-13 04:00:00+00:00


In [45]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  1 non-null      int32                  
 1   predicted_demand    1 non-null      float64                
 2   pickup_hour         1 non-null      datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 152.0 bytes


In [145]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-02-13 17:49:58,661 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 17:49:58,717 INFO: Initializing external client
2025-02-13 17:49:58,720 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 17:50:00,330 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 
Current UTC time: 2025-02-13 22:49:58.659493+00:00
Next hour: 2025-02-13 23:00:00+00:00
Querying for date range: 2025-02-13 to 2025-02-14
Filtering for hour: 2025-02-13 23:00
Found 0 records


In [127]:
now = datetime.now(timezone.utc)

In [141]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
256,168,1.0,2025-02-13 05:00:00+00:00
257,197,0.0,2025-02-13 05:00:00+00:00
258,169,0.0,2025-02-13 05:00:00+00:00
259,209,11.0,2025-02-13 05:00:00+00:00
260,196,0.0,2025-02-13 05:00:00+00:00
...,...,...,...
507,166,13.0,2025-02-13 05:00:00+00:00
508,17,2.0,2025-02-13 05:00:00+00:00
509,31,0.0,2025-02-13 05:00:00+00:00
510,259,0.0,2025-02-13 05:00:00+00:00


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  256 non-null    int32                  
 1   predicted_demand    256 non-null    float64                
 2   pickup_hour         256 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 5.1 KB


In [17]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [27]:
dt = current_date.ceil('h')


In [107]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-02-12 23:05:33,600 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-12 23:05:33,616 INFO: Initializing external client
2025-02-12 23:05:33,616 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-12 23:05:34,834 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.33s) 


In [124]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.33s) 


,pickup_location_id,predicted_demand,pickup_hour
0,22,0.0,2025-02-13 04:00:00+00:00
1,131,0.0,2025-02-13 04:00:00+00:00
2,36,0.0,2025-02-13 04:00:00+00:00
3,83,0.0,2025-02-13 04:00:00+00:00
4,152,1.0,2025-02-13 04:00:00+00:00
...,...,...,...
251,173,0.0,2025-02-13 04:00:00+00:00
252,229,61.0,2025-02-13 04:00:00+00:00
253,208,0.0,2025-02-13 04:00:00+00:00
254,45,4.0,2025-02-13 04:00:00+00:00


In [108]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,22,0.0,2025-02-13 04:00:00+00:00
1,131,0.0,2025-02-13 04:00:00+00:00
2,36,0.0,2025-02-13 04:00:00+00:00
3,83,0.0,2025-02-13 04:00:00+00:00
4,152,1.0,2025-02-13 04:00:00+00:00
...,...,...,...
251,173,0.0,2025-02-13 04:00:00+00:00
252,229,61.0,2025-02-13 04:00:00+00:00
253,208,0.0,2025-02-13 04:00:00+00:00
254,45,4.0,2025-02-13 04:00:00+00:00


In [161]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [162]:
predictions = fetch_next_hour_predictions()

2025-02-13 17:56:01,671 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 17:56:01,681 INFO: Initializing external client
2025-02-13 17:56:01,684 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 17:56:03,247 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551
Current UTC time: 2025-02-13 22:56:01.671315+00:00
Next hour: 2025-02-13 23:00:00+00:00
Found 256 records


In [155]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-02-13 17:53:22,384 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 17:53:22,397 INFO: Initializing external client
2025-02-13 17:53:22,398 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 17:53:23,798 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551


In [156]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.84s) 


In [158]:
df["pickup_hour"].max()

Timestamp('2025-02-13 23:00:00+0000', tz='Etc/UTC')

In [159]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-02-13 23:00:00+00:00


In [160]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
34594,211,29.0,2025-02-13 23:00:00+00:00
34595,53,0.0,2025-02-13 23:00:00+00:00
34596,112,0.0,2025-02-13 23:00:00+00:00
34598,6,0.0,2025-02-13 23:00:00+00:00
34599,240,0.0,2025-02-13 23:00:00+00:00
...,...,...,...
34905,153,0.0,2025-02-13 23:00:00+00:00
34906,187,0.0,2025-02-13 23:00:00+00:00
34907,237,272.0,2025-02-13 23:00:00+00:00
34908,251,0.0,2025-02-13 23:00:00+00:00
